In [1]:
import recpack

In [30]:
import pandas as pd
import gcsfs
import numpy as np

In [3]:
dataframe = pd.read_csv(
    'gs://froomle-playground-robin-1-data/datasets/ML_100K/u.data',
    sep='\t',
    header=None,
    names = ['userId', 'movieId', 'rating', 'timestamp'])

/Users/robinverachtert/workspace/doctorate/dist-env/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
dataframe

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [5]:
from recpack import algorithms

In [6]:
from recpack import helpers

In [7]:
from recpack import evaluate

In [8]:
item_ids = list(dataframe['movieId'].unique())
user_ids = list(dataframe['userId'].unique())

In [9]:
item_id_mapping = helpers.rescale_id_space(item_ids)

In [10]:
user_id_mapping = helpers.rescale_id_space(user_ids)

In [11]:
dataframe['iid'] = dataframe['movieId'].map(lambda x: item_id_mapping[x])
dataframe['uid'] = dataframe['userId'].map(lambda x: user_id_mapping[x])

In [12]:
df = dataframe.drop(['userId', 'movieId', 'rating', 'timestamp'], axis=1)

In [13]:
# Still need to move over the train test split function. 

In [14]:
sp_mat = helpers.create_csr_matrix_from_pandas_df(df, 'iid', 'uid')

In [34]:
algo = algorithms.get_algorithm('ease')

In [35]:
A = algo()

In [17]:
A.fit(sp_mat)

matrix([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.01256535, ...,  0.00019927,
          0.00019927,  0.00019927],
        [ 0.        ,  0.01184177,  0.        , ..., -0.00024994,
         -0.00024994, -0.00024994],
        ...,
        [ 0.        ,  0.00021674, -0.00028848, ...,  0.        ,
          0.00074306,  0.00074306],
        [ 0.        ,  0.00021674, -0.00028848, ...,  0.00074306,
          0.        ,  0.00074306],
        [ 0.        ,  0.00021674, -0.00028848, ...,  0.00074306,
          0.00074306,  0.        ]])

In [ ]:
# Recommendability Idea

In [26]:
pop_list = sorted(list(range(sp_mat.shape[1])), key=lambda x: sp_mat.getcol(x).sum(), reverse=True)[:50]

In [31]:
weights = [1 if i in pop_list else 0 for i in list(range(sp_mat.shape[1]))]
np.array(weights).sum()

50

In [37]:
A.fit(sp_mat, w=np.array(weights))

matrix([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.01256535, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        , -0.00028848, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        , -0.00028848, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        , -0.00028848, ...,  0.        ,
          0.        ,  0.        ]])

In [18]:
import os

In [19]:
os.getcwd()

'/Users/robinverachtert/workspace/doctorate/recpack/notebooks'

In [19]:
# A.save(f"{os.getcwd()}/B")

In [20]:
A.load(f"{os.getcwd()}/B.npy")

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 2.73249368e-03, ...,
        1.80252013e-04, 1.80252013e-04, 1.80252013e-04],
       [0.00000000e+00, 2.91595492e-03, 0.00000000e+00, ...,
        8.58224121e-06, 8.58224121e-06, 8.58224121e-06],
       ...,
       [0.00000000e+00, 1.98520080e-04, 8.85734153e-06, ...,
        0.00000000e+00, 5.35271072e-04, 5.35271072e-04],
       [0.00000000e+00, 1.98520080e-04, 8.85734153e-06, ...,
        5.35271072e-04, 0.00000000e+00, 5.35271072e-04],
       [0.00000000e+00, 1.98520080e-04, 8.85734153e-06, ...,
        5.35271072e-04, 5.35271072e-04, 0.00000000e+00]])

In [38]:
metric1 = evaluate.NDCGK(20)

In [39]:
metric2 = evaluate.RecallK(20)

In [40]:
evaluator = evaluate.FoldInPercentage(0.8, sp_mat)

In [41]:
for _in, _out in evaluator:
    
    if _in.nnz == 0 or _out.nnz == 0:
        print("oops")
        continue
        
    
    
    X_pred = A.predict(_in)
    
    metric1.update(X_pred[0], _out)
    metric2.update(X_pred[0], _out)


IndexError: index 844 is out of bounds for axis 0 with size 1

In [33]:
metric1.value

0.2766473793137888

In [34]:
metric2.value

0.29331242051038564